# 05 — Hedging Portfolio with VIX Futures

In this notebook we investigate how to hedge exposure to equities using the VIX.

Exposure to the VIX index is in general hard to achieve for investers. It can be achieved through VIX options, futures (provided by CBOE) or ETFs that track VIX futures. We use a short term ETF that tracks the **S&P 500 VIX Short-Term Futures index** called **iPath Series B S&P 500 VIX Short-Term Futures ETN (VXX)**. This is inappropriate for long-term hedged positions i.e. buy-and-hold portfolios.

The fund tracks an index with exposure to futures contracts on the CBOE Volatility Index with average one-month maturity and exposure resets daily.

VXX was launched on Jan 19, 2018 and is issued by iPath.

One reason for this choice is the ease of data collection for this particular ETF (we can scrape Yahoo Finance).

**ETF Flow Analysis Report**:

"It’s important to note that VXX does not represent a spot investment in the VIX, but rather is linked to an index comprised of VIX futures. As such, the performance of this product will often vary significantly from a hypothetical investment in the VIX (which isn’t possible to establish). The focus on short-dated futures increases the correlation to the VIX, but also increases the potential for the adverse impacts of contango. Longer-dated options such as VIIZ, VIXM, or VXZ may be appropriate for longer holding period."

"One structural note: as an ETN, VXX avoids tracking error but may expose investors to credit risk, as well as unique tax treatments. VIXY offers similar exposure in an ETF wrapper, while VIIX is a near-identical ETN alternative."

References:
- https://www.cboe.com/tradable-products/vix/vix-futures
- https://ipathetn.cib.barclays/details.app;instrumentId=341408
- https://www.marketbeat.com/stocks/BATS/VXX/
- https://etfdb.com/etf/VXX/#etf-ticker-profile
